In [ ]:
!pip install -q anthropic

In [ ]:
import os
try:
    from google.colab import userdata
    os.environ["ANTHROPIC_API_KEY"] = userdata.get("ANTHROPIC_API_KEY")
except ImportError:
    pass  # Not running in Colab; uses env var already set


# Exercise 4: Tool Definition

In this exercise, you'll define a tool and see how Claude asks to use it.

By the end, you'll understand:
- How to define a tool using a JSON schema
- That Claude doesn't **call** tools — it returns a `tool_use` content block **asking you** to call them
- The structure of a `tool_use` response

## Step 1: Setup

In [ ]:
import anthropic
import json

client = anthropic.Anthropic()

## Step 2: Define a tool

A tool definition has three parts:
- `name` — a unique identifier
- `description` — tells Claude when and how to use it
- `input_schema` — a JSON Schema describing the expected parameters

In [ ]:
tools = [
    {
        "name": "get_weather",
        "description": "Get the current weather in a given location",
        "input_schema": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                }
            },
            "required": ["location"],
        },
    }
]

## Step 3: Send a message that should trigger tool use

When we ask about the weather, Claude should recognize that it needs to use the `get_weather` tool.

In [ ]:
messages = [{"role": "user", "content": "What is the weather in Tokyo?"}]

message = client.messages.create(
    model="claude-haiku-4-5-20251001",
    max_tokens=1024,
    messages=messages,
    tools=tools,
)

## Step 4: Inspect the response

Notice that `stop_reason` is `"tool_use"`, not `"end_turn"`. The model is **pausing** and asking us to run a tool.

In [ ]:
print(json.dumps(json.loads(message.to_json()), indent=2))

Look at `message.content` — it contains a `tool_use` block with:
- `id` — a unique ID for this tool call (you'll need this later)
- `name` — which tool Claude wants to use
- `input` — the arguments Claude extracted from the user's message

In [ ]:
for block in message.content:
    print(f"Type: {block.type}")
    if block.type == "tool_use":
        print(f"  Tool: {block.name}")
        print(f"  Input: {block.input}")
        print(f"  ID: {block.id}")

## Key takeaway

Claude never actually calls your tool. It tells you **which** tool to call and **what arguments** to pass. It's your job to execute the tool and send the result back. That's what the next exercise covers.